time series forecasting mode

Train the ARIMA Model

In [118]:
# Load the sales dataset 
sales_data = pd.read_csv("Pizza_sales.csv")



In [119]:
ingredients_data = pd.read_csv("Pizza_ingredients.csv")

In [120]:
# Date Parsing(handle different date formats for order_date)
def parse_dates(date):
  for fmt in ('%d-%m-%Y', '%d/%m/%Y'):
    try:
      return pd.to_datetime(date, format=fmt)
    except ValueError:
      pass
  raise ValueError(f'no valid date format found for {date}')

In [121]:
sales_data['order_date'] = sales_data['order_date'].apply(parse_dates)
sales_data = sales_data[['order_date', 'pizza_name', 'quantity']]

In [122]:
sales_summary = sales_data.groupby(['order_date', 'pizza_name']).sum().reset_index()
sales_summary.head()

,order_date,pizza_name,quantity
0,2015-01-01,The Barbecue Chicken Pizza,11
1,2015-01-01,The Big Meat Pizza,5
2,2015-01-01,The Calabrese Pizza,1
3,2015-01-01,The California Chicken Pizza,4
4,2015-01-01,The Chicken Alfredo Pizza,2


In [123]:
# reshape data for time series modeling
sales_pivot = sales_summary.pivot(index='order_date', columns='pizza_name', values='quantity').fillna(0)
sales_pivot.head()

pizza_name,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
order_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,11.0,5.0,0.0,1.0,4.0,2.0,4.0,10.0,7.0,6.0,...,4.0,4.0,1.0,7.0,5.0,2.0,5.0,2.0,8.0,3.0
2015-01-02,9.0,8.0,0.0,0.0,5.0,1.0,2.0,6.0,6.0,3.0,...,4.0,6.0,4.0,6.0,11.0,2.0,8.0,4.0,9.0,5.0
2015-01-03,7.0,8.0,1.0,1.0,12.0,5.0,3.0,5.0,4.0,7.0,...,7.0,7.0,1.0,5.0,4.0,6.0,2.0,2.0,10.0,7.0
2015-01-04,11.0,0.0,2.0,2.0,5.0,5.0,1.0,4.0,4.0,4.0,...,5.0,3.0,0.0,1.0,6.0,3.0,4.0,4.0,5.0,1.0
2015-01-05,2.0,3.0,2.0,0.0,8.0,9.0,2.0,9.0,6.0,5.0,...,3.0,5.0,2.0,4.0,2.0,4.0,2.0,3.0,6.0,2.0


In [124]:
from statsmodels.tsa.arima.model import ARIMA

arima_models = {}

for pizza_name in sales_pivot.columns:
  try:
    model = ARIMA(sales_pivot[pizza_name], order=(1, 1, 0))
    model_fit = model.fit()
    arima_models[pizza_name] = model_fit
  except:
    print(f'ARIMA model for {pizza_name} failed to fit')


C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency in

In [125]:
# Generate predictions for one week
prediction_days = 7
predictions_arima = {}

for pizza_name, model in arima_models.items():
  predictions_arima[pizza_name] = model.predict(start=len(sales_pivot), end=len(sales_pivot) + prediction_days - 1)

C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\navee\Desktop\savi\capstone5\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer inde

In [126]:
predictions_df = pd.DataFrame(predictions_arima)


In [127]:
# sets the prediction 7 days
predictions_df.index = pd.date_range(start=sales_pivot.index[-1], periods=prediction_days, freq='D')

In [128]:
ingredients_df = ingredients_data[['pizza_name', 'pizza_ingredients', 'Items_Qty_In_Grams']]

In [129]:
# change column name to items_qty
ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)

C:\Users\navee\AppData\Local\Temp\ipykernel_1752\1198205954.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)


In [130]:
predictions_df

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
2015-12-31,7.940297,4.420315,0.868596,1.368239,7.0,2.529125,2.139860,6.884102,3.033661,6.965904,...,3.595451,2.583928,2.064211,5.918578,5.804710,3.537620,2.0,3.169020,3.0,7.773728
2016-01-01,9.000891,5.252117,1.508634,2.255787,7.0,3.250283,3.004890,8.502239,4.000283,8.000291,...,4.253037,3.751409,2.502062,5.496685,7.263264,4.250472,2.0,4.007142,3.0,8.764985
2016-01-02,8.454764,4.814122,1.146563,1.773032,7.0,2.896705,2.602621,7.661914,3.525106,7.474280,...,3.945166,3.187265,2.297193,5.690456,6.597478,3.902985,2.0,3.623496,3.0,8.323624
2016-01-03,8.735979,5.044753,1.351387,2.035612,7.0,3.070061,2.789690,8.098308,3.758696,7.741769,...,4.089306,3.459867,2.393050,5.601459,6.901389,4.072371,2.0,3.799108,3.0,8.520142
2016-01-04,8.591174,4.923311,1.235518,1.892789,7.0,2.985066,2.702696,7.871682,3.643867,7.605745,...,4.021822,3.328142,2.348199,5.642335,6.762663,3.989802,2.0,3.718722,3.0,8.432641
2016-01-05,8.665738,4.987258,1.301065,1.970474,7.0,3.026739,2.743152,7.989373,3.700315,7.674916,...,4.053417,3.391794,2.369185,5.623561,6.825988,4.030051,2.0,3.755518,3.0,8.471601
2016-01-06,8.627343,4.953586,1.263985,1.928219,7.0,3.006307,2.724338,7.928254,3.672566,7.639741,...,4.038625,3.361036,2.359366,5.632183,6.797082,4.010431,2.0,3.738675,3.0,8.454254


In [131]:
ingredients_df.head()

,pizza_name,pizza_ingredients,items_qty
0,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,The Barbecue Chicken Pizza,Red Peppers,15.0
2,The Barbecue Chicken Pizza,Green Peppers,20.0
3,The Barbecue Chicken Pizza,Tomatoes,30.0
4,The Barbecue Chicken Pizza,Red Onions,60.0


In [132]:
# Create a dictionary to store the ingredient quantities
ingredient_quantities = {}

# Iterate through each pizza in the predictions
for pizza_name in predictions_df.columns:
  # Get the predicted quantity for the pizza
  predicted_quantity = predictions_df[pizza_name].sum()

  # Get the ingredients for the pizza
  pizza_ingredients = ingredients_df[ingredients_df['pizza_name'] == pizza_name]

  # Iterate through each ingredient for the pizza
  for index, row in pizza_ingredients.iterrows():
    ingredient = row['pizza_ingredients']
    ingredient_qty = row['items_qty']

    # Calculate the required quantity of the ingredient
    required_quantity = predicted_quantity * ingredient_qty

    # Add the required quantity to the dictionary
    if ingredient not in ingredient_quantities:
      ingredient_quantities[ingredient] = 0
    ingredient_quantities[ingredient] += required_quantity

In [133]:
# Create a DataFrame from the ingredient quantities
ingredient_requirements_df = pd.DataFrame.from_dict(ingredient_quantities, orient='index', columns=['required_quantity'])

In [134]:
ingredient_requirements_df

,required_quantity
Barbecued Chicken,5401.456838
Red Peppers,11339.700349
Green Peppers,8028.578723
Tomatoes,34970.259430
Red Onions,54801.315406
...,...
Luganega Sausage,2767.612911
Onions,1383.806456
Soppressata Salami,4899.979682
Peperoncini verdi,1408.577241


In [135]:
# Create a purchase order DataFrame
purchase_order_df = ingredient_requirements_df.copy()

# Add a column for the unit of measure (assuming all ingredients are in grams)
purchase_order_df['unit'] = 'grams'

# Rename the columns for better readability
purchase_order_df = purchase_order_df.rename(columns={'required_quantity': 'quantity'})

# Add a header to the purchase order
print('Purchase Order:')
print('----------------')

# Print the purchase order table
print(purchase_order_df.to_string())

Purchase Order:
----------------
                                quantity   unit
Barbecued Chicken            5401.456838  grams
Red Peppers                 11339.700349  grams
Green Peppers                8028.578723  grams
Tomatoes                    34970.259430  grams
Red Onions                  54801.315406  grams
Barbecue Sauce               1800.485613  grams
Bacon                       19992.004764  grams
Pepperoni                   24186.495331  grams
Italian Sausage               343.954622  grams
Chorizo Sausage              1719.773109  grams
Brie Carre Cheese             260.272404  grams
Prosciutto                    260.272404  grams
Caramelized Onions                   NaN  grams
Pears                          86.757468  grams
Thyme                          43.378734  grams
Garlic                      17931.797062  grams
?duja Salami                 1586.898271  grams
Pancetta                     2380.347406  grams
Friggitello Peppers           396.724568  grams
Chicken

In [136]:
purchase_order_df.to_csv("purchase_order.csv")